In [1]:
import redis
import pickle 
import torch
database = redis.Redis(host='localhost', port=6379)

In [ ]:
import glob
from tqdm import tqdm
import torch
feature_dirs = glob.glob('/remote-home/share/DATA/brca-4096-processed/features/pt_files/*.pt')
features = [] 
for x in tqdm(feature_dirs):
    features.append(torch.load(x))

### load data into redis for pretraining [384, 16, 16]

In [6]:
import redis
redis_database = redis.Redis(host='localhost', port=6379)
keys = redis_database.keys()
max_key = max([int(x) for x in keys])
print(max_key)

101040


In [ ]:
from tqdm import tqdm
import pickle
import json
import redis
database = redis.Redis(host='localhost', port=6379)
feature_ind_map = {}
dataset_ind = 62852 + 1
ind = 0
for wsi_ind, (feature_dir, feature) in tqdm(enumerate(zip(feature_dirs, features))):
    slide_id = feature_dir.split('/')[-1][:-3] # e.g. 'TCGA-85-7698-01Z-00-DX1.bc8a89a7-356e-4913-9632-627774730f48'
    region_num_this_wsi = len(feature)
    
    for region_ind in range(region_num_this_wsi):
        region_feature = feature[region_ind] # Tensor, [384, 16, 16]
        feature_ind_map[ind] = {'slide_id': slide_id, 'slide_index': wsi_ind, 'region_index': region_ind, 'dataset_index': dataset_ind}
        # region_feature_bytes = encode_wb(region_feature)
        region_feature_bytes = pickle.dumps(region_feature.numpy())
        database.set(dataset_ind, region_feature_bytes)
        dataset_ind += 1
        ind += 1
print(dataset_ind)
with open('tcga_brca_4k.json','w', encoding='utf8') as f:
    json.dump(feature_ind_map, f,   ensure_ascii=False,  indent=2)
    


### load data into redis for downsteam tasks [N, 384, 16, 16]

In [ ]:
from tqdm import tqdm
import pickle
import json
import redis
database = redis.Redis(host='localhost', port=6379)
feature_ind_map = {}
dataset_ind = 200000 # 360154
ind = 0
for wsi_ind, (feature_dir, feature) in tqdm(enumerate(zip(feature_dirs, features))):
    slide_id = feature_dir.split('/')[-1][:-3] # e.g. 'TCGA-85-7698-01Z-00-DX1.bc8a89a7-356e-4913-9632-627774730f48'
    feature_ind_map[ind] = {'slide_id': slide_id, 'slide_index': wsi_ind, 'dataset_index': dataset_ind}
    feature_bytes = pickle.dumps(feature.numpy())
    database.set(dataset_ind, feature_bytes)
    dataset_ind += 1
    ind += 1
    del feature
print(dataset_ind)
with open('tcga_brca_WSI.json','w', encoding='utf8') as f:
    json.dump(feature_ind_map, f,   ensure_ascii=False,  indent=2)